In [ ]:
"""
# Copyright (C) 2024 Jesús Bautista Villar <jesbauti20@gmail.com>
"""
!python3 -V || python -V

Python 3.11.4


In [2]:
from _requirements import *

## Diagonalization of the affine transformation matrix $A$ $(2\times2)$

In [3]:
a = 2
omega = 1
hx = 0
hy = 0

A = a*np.eye(2) + omega*np.array([[0,-1],[1,0]]) + np.array([[0,hx],[hy,0]])
print("A =\n",A,"\n")

# Predicted eigenvalues and eigenvectors
lambda1 = a + np.sqrt((hx - omega)*(hy + omega) + 0j)
lambda2 = a - np.sqrt((hx - omega)*(hy + omega) + 0j)

if hx-omega == 0 and hy+omega == 0: # The matrix is already diagonal
    v1 = np.array([1, 0])
    v2 = np.array([0, 1])
elif hx-omega != 0 and hy+omega !=0: # Diagonalizable
    a = np.sqrt(hx - omega + 0j)
    b = np.sqrt(hy + omega + 0j)
    denom = np.sqrt(a*np.conj(a) + b*np.conj(b))

    v1 = np.array([a,  b])/denom
    v2 = np.array([a, -b])/denom
elif hx-omega != 0: # Not diagonalizable (we need 2 generalized eigenvectors)
    v1 = np.array([1, 0])
    v2 = np.array([0, 1/(hx-omega)])
else: # (hy+omega != 0) Not diagonalizable (we need 2 generalized eigenvectors)
    v1 = np.array([0, 1])
    v2 = np.array([1/(hy+omega), 0])

V = np.array([v1,v2]).T

print("------- Predicted")
with np.printoptions(precision=3, suppress=True):
    print("lambda_1 = {:.3f} -- ".format(lambda1), v1)
    print("lambda_2 = {:.3f} -- ".format(lambda2), v2)

# Comparing with the computed eigen values and eigenvectors
print("------- Computed")
lambdas, vs = LA.eig(A)
with np.printoptions(precision=3, suppress=True):
    print("lambdas: ", lambdas)
    print("v1 = ", vs[:,0])
    print("v2 = ", vs[:,1])

A =
 [[ 2. -1.]
 [ 1.  2.]] 

------- Predicted
lambda_1 = 2.000+1.000j --  [0.   +0.707j 0.707+0.j   ]
lambda_2 = 2.000-1.000j --  [ 0.   +0.707j -0.707+0.j   ]
------- Computed
lambdas:  [2.+1.j 2.-1.j]
v1 =  [0.707+0.j    0.   -0.707j]
v2 =  [0.707-0.j    0.   +0.707j]


## Diagonalization of $\tilde L = (L - MB^\top)$ in $\mathbb{C}$

In [4]:
def affine_eig_complex(sim_fr, vx, vy, a, omega, hx, hy):
    kappa = sim_fr.kappa
    h = sim_fr.h
    n = sim_fr.n

    # Complex conjugate eigen values
    sigma1 = np.sqrt((hx-omega)*(hy+omega) + 0j)
    sigma2 = - np.sqrt((hx-omega)*(hy+omega) + 0j)
    l1 = kappa*(a + sigma1)
    l2 = kappa*(a + sigma2)

    # Compute the eigenvects using the analytical expressions (5 cases)
    with np.printoptions(precision=3, suppress=True):
        print("----- M @ B.T")
        print(sim_fr.M@sim_fr.B.T, "\n")
        
        ps_x = np.real(sim_fr.p_star_c)
        ps_y = np.imag(sim_fr.p_star_c)

        print("l+ =", l1, "l- =", l2) 

        # ##### Eigenvectors ##################################################
        print("\n----- Eigenvector of L_mod")

        # (l1 != 0 and l2 != 0) -----------------------------------------------
        if l1 != 0 and l2 != 0:
            print("(l1 != 0, l2 != 0) \n")

            gamma1 = (vx*(hy + omega) + vy*(sigma1))/(l1/kappa)
            gamma2 = (vx*(hy + omega) + vy*(sigma2))/(l2/kappa)

            print("gamma1 =", gamma1)
            print("gamma2 =", gamma2, "\n")

            v0 = np.ones(4)
            v1 = (hy + omega)*ps_x + sigma1*ps_y + np.ones(4)*gamma1
            v2 = (hy + omega)*ps_x + sigma2*ps_y + np.ones(4)*gamma2

            print("v1(l+) = ", v1)
            print("v2(l-) = ", v2)
            print("v1(l+)_unit = ", unit_vec(v1))
            print("v2(l-)_unit = ", unit_vec(v2), "\n")
     

            print("----")
            print("test-v0     | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T - np.eye(n)*0 ) @ v0)
            print("test-v1(l+) | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T - np.eye(n)*l1) @ v1)
            print("test-v2(l-) | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T - np.eye(n)*l2) @ v2)
            print("----")
            print("test-v0     | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n)*0 ) @ v0)
            print("test-v1(l+) | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n)*l1) @ v1)
            print("test-v2(l-) | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n)*l2) @ v2)
        # ---------------------------------------------------------------------

        # (l1 != 0 or l2 != 0) ------------------------------------------------
        elif l1 != 0 or l2 != 0:

            # A: (l+ = 0)
            if l1 == 0:
                print("A case - (l+ == 0) \n")
                l_deg = l1

                v1 = np.ones(4)

                gamma2 = (vx*(hy + omega) + vy*(sigma2))/(l2/kappa)  # = -(vx-vy) / (l2/kappa)
                v2 = (hy + omega)*ps_x + sigma2*ps_y + np.ones(4)*gamma2

            # B: (l- = 0)
            else:
                print("B case - (l- == 0) \n")
                l_deg = l2

                gamma1 = (vx*(hy + omega) + vy*(sigma1))/(l1/kappa) 
                v1 = (hy + omega)*ps_x + sigma1*ps_y + np.ones(4)*gamma1

                v2 = np.ones(4)
            
            v_deg = ((hy + omega)*ps_x - a*ps_y)
            #v_deg = ((hx - omega)*ps_y - a*ps_x)

            print("v1(l+) = ", v1)
            print("v2(l-) = ", v2)
            print("v_deg = ", v_deg)
            print("v1(l+)_unit = ", unit_vec(v1))
            print("v2(l+)_unit = ", unit_vec(v2))
            print("v_deg_unit = ", unit_vec(v_deg), "\n")

            print("----")
            print("test-v1(l+) | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T - np.eye(n)*l1) @ v1)
            print("test-v2(l-) | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T - np.eye(n)*l2) @ v2)
            print("----")
            print("test-v1(l+) | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n)*l1) @ v1)
            print("test-v2(l-) | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n)*l2) @ v2)
            print("----")
            print("test-v_deg  | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T - np.eye(n)*l_deg) @ v_deg)
            print("test-v_deg  | (kappa*M@B.T)^2:", (kappa*sim_fr.M@sim_fr.B.T - np.eye(n)*l_deg)@(kappa*sim_fr.M@sim_fr.B.T - np.eye(n)*l_deg) @ v_deg)
        # ---------------------------------------------------------------------

        # Degenerated cases (l+- = 0) #########################################

        # (l+- = 0) ----- (hx != 0 and vy != 0) or (hy != 0 and vx != 0)
        elif (hx != 0 and vy != 0) or (hy != 0 and vx != 0):
            print("(l+- = 0) ----- (hx != 0 and vy != 0) or (hy != 0 and vx != 0) \n")
            
            v1 = np.ones(4)

            v_deg = sim_fr.p_star_c

            print("v1 =", v1)
            print("v_deg = ", v_deg)
            print("v1_unit = ", unit_vec(v1))
            print("v_deg_unit = ", unit_vec(v_deg), "\n")

            print("----")
            print("test-v1 | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T - np.eye(n)*0) @ v1)
            print("----")
            print("test-v1 | -h*L_mod:", -h*sim_fr.L_mod @ v1)
            print("----")
            print("test-v_deg | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T) @ v_deg)
            print("test-v_deg | (kappa*M@B.T)^2:", (kappa*sim_fr.M@sim_fr.B.T)@(kappa*sim_fr.M@sim_fr.B.T) @ v_deg)
        # ---------------------------------------------------------------------

        # (l+- = 0)  ----- (hx != 0 and vx != 0) or (hy != 0 and vy != 0)
        elif (hx != 0 or vy == 0) or (hy != 0 or vx == 0):
            print("(l+- = 0)  ----- (hx != 0 and vx != 0) or (hy != 0 and vy != 0) \n")

            if hx != 0:
                v1 = vx*np.ones(4) + hx*ps_y
                v2 = np.ones(4) - (vx/hx)*ps_y
            else:
                v1 = 1j*(vy*np.ones(4) + hy*ps_x)
                v2 = 1j*(np.ones(4) - (vy/hy)*ps_x)
                
            v_deg = sim_fr.p_star_c

            print("v1 = ", v1)
            print("v2 = ", v2)
            print("v_deg = ", v_deg)
            print("v1_unit = ", unit_vec(v1))
            print("v2_unit = ", unit_vec(v2))
            print("v_deg_unit = ", unit_vec(v_deg), "\n")

            print("----")
            print("test-v1 | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T) @ v1)
            print("test-v2 | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T) @ v2)
            print("----")
            print("test-v1 | -h*L_mod:", (-h*sim_fr.L_mod) @ v1)
            print("test-v2 | -h*L_mod:", (-h*sim_fr.L_mod) @ v2)
            print("----")
            print("test-v_deg | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T) @ v_deg)
        # ---------------------------------------------------------------------

        # (l+- = hx = hy = 0) ----- vx,vy != 0
        else:
            print("C1: (l+- = hx = hy = 0) ----- vx,vy != 0 \n")

            v1 = (vx)*ps_y - (vy)*ps_x
            v2 = np.ones(4, dtype=complex)

            v_deg = sim_fr.p_star_c

            print("v1 = ", v1/LA.norm(v1))
            print("v2 = ", v2/LA.norm(v2))
            print("v_deg = ", v_deg/LA.norm(v_deg), "\n")

            print("----")
            print("test-v1 | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T) @ v1)
            print("test-v2 | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T) @ v2)
            print("----")
            print("test-v1 | -h*L_mod:", (-h*sim_fr.L_mod) @ v1)
            print("test-v2 | -h*L_mod:", (-h*sim_fr.L_mod) @ v2)
            print("----")
            print("test-v_deg | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B.T) @ v_deg)
        # ---------------------------------------------------------------------

### Some symbolic computations

In [5]:
import sympy as sp
from sympy.matrices import Matrix

# Determinant symbolic computation (degenerated case)
hx,vy,l = sp.symbols('hx,vy,l')

H = Matrix([[hx-vy-l, vy        , 0      , -hx       ], 
            [-vy    , -(hx-vy)-l, hx     , 0         ],
            [0      , -hx       , hx+vy-l, -vy       ],
            [hx     , 0         , vy     , -(hx+vy)-l]])

H.det()

l**4

### Square

In [6]:
## Parameters #####
n = 4
Z = ((1,2), (1,3), (2,3), (2,4), (3,4), (4,1))
p_star = np.array([[-1,-1], [-1,1], [1,1], [1,-1]]) # square
p0 = [[-10,-3], [-5,-2], [10,4], [0,-5]]
tf = 1
h = 4
kappa = 0.01
K = np.eye(n)

## Generate the simulation frame #####
args = {"Z":Z, "p_star":p_star, "p0":p0, "tf":tf, "h":h, "kappa":kappa, "K":K}
sim_fr = AffineComplexSimulator(**args)

## Debug --
sim_fr.check_W_L()
sim_fr.check_kernel()

Failed to dualize the problem: 'float' object is not iterable
W (6, 6) = 
 [[ 0.25  0.    0.    0.    0.    0.  ]
 [ 0.   -0.25  0.    0.    0.    0.  ]
 [ 0.    0.    0.25  0.    0.    0.  ]
 [ 0.    0.    0.   -0.25  0.    0.  ]
 [ 0.    0.    0.    0.    0.25  0.  ]
 [ 0.    0.    0.    0.    0.    0.25]]
L (4, 4) = 
 [[ 0.25 -0.25  0.25 -0.25]
 [-0.25  0.25 -0.25  0.25]
 [ 0.25 -0.25  0.25 -0.25]
 [-0.25  0.25 -0.25  0.25]]
 --- Eigenvalues
lambda_0 = -0.000000+0.000000j
lambda_1 = -1.000000+0.000000j
lambda_2 = 0.000000+0.000000j
lambda_3 = -0.000000+0.000000j
--- Eigenvectors
v_0 = [ 0.866+0.j  0.289-0.j -0.289-0.j  0.289-0.j]
v_1 = [-0.5-0.j  0.5-0.j -0.5-0.j  0.5+0.j]
v_2 = [0.   +0.j 0.577+0.j 0.789+0.j 0.211+0.j]
v_3 = [ 0.564-0.j  0.618+0.j -0.36 -0.j -0.413-0.j]
--- Kernel Eigenvectors
L@1_n           =  [0. 0. 0. 0.]
L@p^*           =  [-0.+0.j -0.+0.j  0.+0.j  0.-0.j]
L@Re(p^*)       =  [-0. -0.  0.  0.]
L@Im(p^*)       =  [ 0.  0.  0. -0.]
 ------------ 


In [7]:
## Design the parameters of a body-fixed desired velocity in S #####

# Parameters examples ----------------------------------
# C1
params_deg_c1 = [1, 2, 0, 0, 0, 0]

# C2
params_deg_c2 = [1, 0, 0, 0, 3, 0]

# C3
params_deg_c3 = [1, 0, 0, 0, 0, 4]

# Non-degenerated
params_nodeg1 = [2, 3, 2, 1, 2, -0.5]
params_nodeg2 = [2, 3, 2, 1, 2, 0]
params_nodeg_a = [1, 2, -1, 1, 2, 0] # "A" case (l+ = 0)
params_nodeg_b = [1, 2, 1, 1, 2, 0]  # "B" case (l- = 0)
params_nodeg_b2 = [1, 2, 3.2, 1, 2, 9.24]  # "B" case (l- = 0)

params_test = [1, 0, 0, 1, 1, 0]
# ------------------------------------------------------

# Set the parameters
vx, vy, a, omega, hx, hy = params_test
#vx, vy, a, omega, hx, hy = params_deg_c3

# Design the weights that gerenates the body-fixed desired velocity v_f^*
mu_matrix_x = np.array([[0,0,0,-1], [0,0,-1,0], [0,1,0,0], [1,0,0,0]])/2
mu_matrix_y = np.array([[0,-1,0,0], [1,0,0,0], [0,0,0,1], [0,0,-1,0]])/2

mu_matrix_a = np.array([[0,1,0,1], [1,0,1,0], [0,1,0,1], [1,0,1,0]])/2
mu_matrix_w = np.array([[0,1,0,-1], [-1,0,1,0], [0,-1,0,1], [1,0,-1,0]])/2
mu_matrix_hx = np.array([[0,0,0,1], [0,0,-1,0], [0,1,0,0], [-1,0,0,0]])/2
mu_matrix_hy = np.array([[0,1,0,0], [-1,0,0,0], [0,0,0,1], [0,0,-1,0]])/2

# Pass the weights to the simulation frame, it will compute M and L_mod
sim_fr.set_manual_mu(vx*mu_matrix_x + vy*mu_matrix_y + a*mu_matrix_a + omega*mu_matrix_w + 
                     hx*mu_matrix_hx + hy*mu_matrix_hy)

## Debug --
sim_fr.check_M(eigenvectors=True, prec=[2,4,3])

M (4, 6) B.T (6, 4) K_inv (4, 4)
mu_ij matrix:
 [[ 0.   0.5  0.  -0.5]
 [-0.5  0.  -0.5  0. ]
 [ 0.   0.5  0.   0.5]
 [ 0.5  0.  -0.5  0. ]]
M:
 [[-0.5 -0.   0.   0.   0.  -0.5]
 [-0.5  0.   0.5 -0.   0.   0. ]
 [ 0.   0.   0.5  0.  -0.5  0. ]
 [ 0.   0.   0.   0.  -0.5 -0.5]]
M@B^T@p_star_c [1.-1.j 1.-1.j 1.+1.j 1.+1.j]

 -> L_mod
 --- Eigenvalues
lambda_0 = -4.000000+0.000000j
lambda_1 = 0.000000+0.000000j
lambda_2 = -0.000000+0.000000j
lambda_3 = -0.000000-0.000000j
--- Eigenvectors
v_0 = [ 0.5  +0.j -0.502+0.j  0.498+0.j -0.5  +0.j]
v_1 = [-0.5+0.j -0.5+0.j -0.5+0.j -0.5+0.j]
v_2 = [-0.5+0.j -0.5+0.j -0.5-0.j -0.5-0.j]
v_3 = [-0.5-0.j -0.5-0.j -0.5+0.j -0.5+0.j]


In [8]:
# Our analytical solution for the eigenvalues and eigenvectors
affine_eig_complex(sim_fr, vx, vy, a, omega, hx, hy)

----- M @ B.T
[[ 0.  -0.5  0.   0.5]
 [ 0.5 -1.   0.5  0. ]
 [ 0.  -0.5  1.  -0.5]
 [-0.5  0.   0.5  0. ]] 

l+ = 0j l- = 0j

----- Eigenvector of L_mod
(l+- = 0)  ----- (hx != 0 and vx != 0) or (hy != 0 and vy != 0) 

v1 =  [0. 2. 2. 0.]
v2 =  [2. 0. 0. 2.]
v_deg =  [-1.-1.j -1.+1.j  1.+1.j  1.-1.j]
v1_unit =  [0.    0.707 0.707 0.   ]
v2_unit =  [0.707 0.    0.    0.707]
v_deg_unit =  [-0.354-0.354j -0.354+0.354j  0.354+0.354j  0.354-0.354j] 

----
test-v1 | kappa*M@B.T: [-0.01 -0.01  0.01  0.01]
test-v2 | kappa*M@B.T: [ 0.01  0.01 -0.01 -0.01]
----
test-v1 | -h*L_mod: [-0.01 -0.01  0.01  0.01]
test-v2 | -h*L_mod: [ 0.01  0.01 -0.01 -0.01]
----
test-v_deg | kappa*M@B.T: [0.01-0.01j 0.01-0.01j 0.01+0.01j 0.01+0.01j]


### Trapezoid

In [9]:
## Parameters #####
n = 4
Z = ((1,2), (1,3), (2,3), (2,4), (3,4), (4,1))
p_star = np.array([[-1,-1], [-2,2], [2,2], [1,-1]]) # square
p0 = [[-10,-3], [-5,-2], [10,4], [0,-5]]
tf = 1
h = 1
kappa = 0.01
K = np.eye(n)

## Generate the simulation frame #####
args = {"Z":Z, "p_star":p_star, "p0":p0, "tf":tf, "h":h, "kappa":kappa, "K":K}
sim_fr = AffineComplexSimulator(**args)

## Debug --
sim_fr.check_W_L()
sim_fr.check_kernel()

Failed to dualize the problem: 'float' object is not iterable
W (6, 6) = 
 [[ 0.2  0.   0.   0.   0.   0. ]
 [ 0.  -0.2  0.   0.   0.   0. ]
 [ 0.   0.   0.1  0.   0.   0. ]
 [ 0.   0.   0.  -0.2  0.   0. ]
 [ 0.   0.   0.   0.   0.2  0. ]
 [ 0.   0.   0.   0.   0.   0.4]]
L (4, 4) = 
 [[ 0.4 -0.2  0.2 -0.4]
 [-0.2  0.1 -0.1  0.2]
 [ 0.2 -0.1  0.1 -0.2]
 [-0.4  0.2 -0.2  0.4]]
 --- Eigenvalues
lambda_0 = -1.000000-0.000000j
lambda_1 = -0.000000+0.000000j
lambda_2 = 0.000000+0.000000j
lambda_3 = -0.000000+0.000000j
--- Eigenvectors
v_0 = [ 0.632+0.j -0.316+0.j  0.316+0.j -0.632+0.j]
v_1 = [-0.463+0.j  0.102+0.j  0.877+0.j -0.075+0.j]
v_2 = [0.591+0.j 0.582-0.j 0.318-0.j 0.459-0.j]
v_3 = [-0.13 -0.j  0.678+0.j -0.34 -0.j -0.639-0.j]
--- Kernel Eigenvectors
L@1_n           =  [0. 0. 0. 0.]
L@p^*           =  [-0.-0.j  0.-0.j  0.+0.j  0.-0.j]
L@Re(p^*)       =  [-0.  0.  0.  0.]
L@Im(p^*)       =  [-0. -0.  0. -0.]
 ------------ 


In [10]:
## Design the parameters of a body-fixed desired velocity in S #####

# Parameters examples ----------------------------------
# C1
params_deg_c1 = [1, 2, 0, 0, 0, 0]

# C2
params_deg_c2 = [1, 0, 0, 0, 3, 0]

# C3
params_deg_c3 = [1, 0, 0, 0, 0, 4]

# Non-degenerated
params_nodeg1 = [2, 3, 2, 1, 2, 1]
params_nodeg2 = [2, 3, 2, 1, 2, 0] # beta close to 0
params_nodeg_a = [1, 2, -1, 1, 2, 0] # "A" case (l- = 0)
params_nodeg_b = [1, 2, 1, 1, 2, 0]  # "B" case (l+ = 0)
# ------------------------------------------------------

# Set the parameters
vx, vy, a, omega, hx, hy = params_nodeg1

# Design the weights that gerenates the body-fixed desired velocity v_f^*
mu_matrix_x = np.array([[0,0,0,-1], [0,0,-1/2,0], [0,1/2,0,0], [1,0,0,0]])/2
mu_matrix_y = np.array([[0,-1,0,-1/2], [1,0,-1/4,0], [0,-1/4,0,1], [-1/2,0,-1,0]])/3

mu_matrix_a = np.array([[0,0,1,0], [0,0,0,2], [2,0,0,0], [0,1,0,0]])/3
mu_matrix_w = np.array([[0,1,0,-1], [-2,0,2,0], [0,-2,0,2], [1,0,-1,0]])/3
mu_matrix_hx = np.array([[0,0,0,1], [0,0,-2/2,0], [0,2/2,0,0], [-1,0,0,0]])/2
mu_matrix_hy = -np.array([[0,-1,0,-1/2], [2,0,-2/4,0], [0,2/4,0,-2], [1/2,0,1,0]])/3

sim_fr.set_manual_mu(vx*mu_matrix_x + vy*mu_matrix_y + a*mu_matrix_a + omega*mu_matrix_w + 
                     hx*mu_matrix_hx + hy*mu_matrix_hy)

## Debug --
sim_fr.check_M(eigenvectors=True, prec=[2,4,3])

M (4, 6) B.T (6, 4) K_inv (4, 4)
mu_ij matrix:
 [[ 0.   -0.33  0.67 -0.67]
 [-0.33  0.   -0.92  1.33]
 [ 1.33  0.42  0.    2.33]
 [-0.33  0.67 -1.67  0.  ]]
M:
 [[ 0.33 -0.67  0.    0.    0.   -0.67]
 [-0.33  0.    0.92 -1.33  0.    0.  ]
 [ 0.    1.33  0.42  0.   -2.33  0.  ]
 [ 0.    0.    0.    0.67 -1.67  0.33]]
M@B^T@p_star_c [-1. -1.j  0. +3.j  8.+11.j  3. +3.j]

 -> L_mod
 --- Eigenvalues
lambda_0 = -1.015000
lambda_1 = 0.034142
lambda_2 = -0.000000
lambda_3 = 0.005858
--- Eigenvectors
v_0 = [ 0.632 -0.324  0.292 -0.64 ]
v_1 = [-0.102  0.126  0.939  0.305]
v_2 = [0.5 0.5 0.5 0.5]
v_3 = [-0.126 -0.912  0.095  0.378]


In [11]:
# Our analytical solution for the eigenvalues and eigenvectors
affine_eig_complex(sim_fr, vx, vy, a, omega, hx, hy)

----- M @ B.T
[[-0.333  0.333 -0.667  0.667]
 [ 0.333  0.083  0.917 -1.333]
 [-1.333 -0.417  4.083 -2.333]
 [ 0.333 -0.667  1.667 -1.333]] 

l+ = (0.03414213562373095+0j) l- = (0.005857864376269049+0j)

----- Eigenvector of L_mod
(l1 != 0, l2 != 0) 

gamma1 = (2.414213562373095+0j)
gamma2 = (-0.41421356237309603+0j) 

v1(l+) =  [-1.   +0.j  1.243+0.j  9.243+0.j  3.   +0.j]
v2(l-) =  [-1.   +0.j -7.243+0.j  0.757+0.j  3.   +0.j]
v1(l+)_unit =  [-0.102+0.j  0.126+0.j  0.939+0.j  0.305+0.j]
v2(l-)_unit =  [-0.126+0.j -0.912+0.j  0.095+0.j  0.378+0.j] 

----
test-v0     | kappa*M@B.T: [0. 0. 0. 0.]
test-v1(l+) | kappa*M@B.T: [ 0.+0.j  0.+0.j -0.+0.j -0.+0.j]
test-v2(l-) | kappa*M@B.T: [-0.+0.j  0.+0.j  0.+0.j -0.+0.j]
----
test-v0     | -h*L_mod: [-0.  0.  0.  0.]
test-v1(l+) | -h*L_mod: [ 0.+0.j  0.+0.j -0.+0.j  0.+0.j]
test-v2(l-) | -h*L_mod: [ 0.+0.j -0.+0.j  0.+0.j -0.+0.j]


## Diagonalization of $\tilde L \otimes I_m = (L - MB^\top) \otimes I_m$ in $\mathbb{R}^2$

In [12]:
def affine_eig_R2(sim_fr, vx, vy, a, omega, hx, hy):
    kappa = sim_fr.kappa
    h = sim_fr.h
    n = sim_fr.n

    # Complex conjugate eigen values
    l1 = kappa*(a + np.sqrt((hx-omega)*(hy+omega) + 0j))
    l2 = kappa*(a - np.sqrt((hx-omega)*(hy+omega) + 0j))

    # Compute the eigenvects using the analytical expressions (5 cases)
    with np.printoptions(precision=3, suppress=True):
        # print("----- M @ B.T")
        # print(sim_fr.M@sim_fr.B.T, "\n")
        
        ps_x = sim_fr.p_star[0::2]
        ps_y = sim_fr.p_star[1::2]

        alfa = ((hx-hy)/2 - omega)
        beta1 = ((hx+hy)/2 - np.sqrt((hx-omega)*(hy+omega) + 0j))
        beta2 = ((hx+hy)/2 + np.sqrt((hx-omega)*(hy+omega) + 0j))

        print("l+ =", l1, "l- =", l2) 
        print("alfa, beta1, beta2 =", alfa, beta1, beta2)

        # ##### Eigenvectors ##################################################
        print("\n----- Eigenvector of L_mod")

        # (l1 != 0 and l2 != 0) -----------------------------------------------
        if l1 != 0 and l2 != 0:
            print("(l1 != 0, l2 != 0) \n")

            gamma1 = (vx*(alfa - beta1) + vy*(alfa + beta1))/(2*l1/kappa)
            gamma2 = (vx*(alfa - beta2) + vy*(alfa + beta2))/(2*l2/kappa)

            print("gamma1 =", gamma1)
            print("gamma2 =", gamma2, "\n")

            v0 = np.ones(4)
            u1 = alfa*(ps_y + ps_x)/2 + beta1*(ps_y - ps_x)/2 + np.ones(4)*gamma1
            u2 = alfa*(ps_y + ps_x)/2 + beta2*(ps_y - ps_x)/2 + np.ones(4)*gamma2

            v1 = np.kron(u1,[1,0])
            v2 = np.kron(u1,[0,1])
            v3 = np.kron(u2,[1,0])
            v4 = np.kron(u2,[0,1])

            print("v1(l+)_1 = ", unit_vec(v1))
            print("v2(l+)_2 = ", unit_vec(v2))
            print("v3(l-)_1 = ", unit_vec(v3))
            print("v4(l-)_2 = ", unit_vec(v4), "\n")

            print("---- kappa*M@B.T@v")
            print("test-v1(l+)_1 | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*l1) @ v1)
            print("test-v2(l+)_2 | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*l1) @ v2)
            print("test-v3(l-)_1 | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*l2) @ v3)
            print("test-v4(l-)_2 | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*l2) @ v4)
            print("---- -h*L_mod@v")
            print("test-v1(l+)_1 | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n*2)*l1) @ v1)
            print("test-v2(l+)_2 | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n*2)*l1) @ v2)
            print("test-v3(l-)_1 | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n*2)*l2) @ v3)
            print("test-v4(l-)_2 | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n*2)*l2) @ v4)

        # Degenerated cases (l+- = 0) #########################################

        # ---------------------------------------------------------------------

        # (l+- = hx = hy = 0) ----- vx,vy != 0
        else:
            print("C1: (a = omega = hx = hy = 0) ----- vx,vy != 0 \n")
            u1 = (vx)*ps_y - (vy)*ps_x
            u2 = np.ones(4, dtype=complex)

            u_deg = sim_fr.p_star_c

            v11 = np.kron(u1,[1,0])
            v12 = np.kron(u1,[0,1])
            v21 = np.kron(u2,[1,0])
            v22 = np.kron(u2,[0,1])

            v_ged_1 = u_deg * np.kron(np.ones(n),[1,0])
            v_ged_2 = u_deg * np.kron(np.ones(n),[0,1])

            print("v1_1 = ", v11/LA.norm(v11))
            print("v1_2 = ", v12/LA.norm(v12))
            print("v2_1 = ", v21/LA.norm(v21))
            print("v2_2 = ", v22/LA.norm(v22), "\n")

            print("---- kappa*M@B.T@v")
            print("test-v1_x | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*0) @ v11)
            print("test-v1_y | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*0) @ v12)
            print("test-v2_x | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*0) @ v21)
            print("test-v2_y | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*0) @ v22)
            print("test-v_dg_x | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*0) @ v_ged_1)
            print("test-v_dg_y | kappa*M@B.T:", (kappa*sim_fr.M@sim_fr.B_bar_T - np.eye(n*2)*0) @ v_ged_2)

            print("---- -h*L_mod@v")
            print("test-v1_x | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n*2)*0) @ v11)
            print("test-v1_y | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n*2)*0) @ v12)
            print("test-v2_x | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n*2)*0) @ v21)
            print("test-v2_y | -h*L_mod:", (-h*sim_fr.L_mod - np.eye(n*2)*0) @ v22)
            print("test-v_dg_x | kappa*M@B.T:", (-h*sim_fr.L_mod - np.eye(n*2)*0) @ v_ged_1)
            print("test-v_dg_y | kappa*M@B.T:", (-h*sim_fr.L_mod - np.eye(n*2)*0) @ v_ged_2)
        # ---------------------------------------------------------------------

### Square

In [13]:
## Parameters #####
n = 4
Z = ((1,2), (1,3), (2,3), (2,4), (3,4), (4,1))
p_star = [[-1,-1], [-1,1], [1,1], [1,-1]] # square
p0 = [[-10,-3], [-5,-2], [10,4], [0,-5]]
tf = 2
h = 4
kappa=0.1
K = np.eye(n)

## Generate the simulation frame #####
args = {"Z":Z, "p_star":p_star, "p0":p0, "tf":tf, "h":h, "kappa":kappa, "K":K}
sim_fr = AffineSimulator(**args)

## Debug --
sim_fr.check_W_L(eigenvectors=False)

Failed to dualize the problem: 'float' object is not iterable
W (6, 6) = 
 [[ 0.25  0.    0.    0.    0.    0.  ]
 [ 0.   -0.25  0.    0.    0.    0.  ]
 [ 0.    0.    0.25  0.    0.    0.  ]
 [ 0.    0.    0.   -0.25  0.    0.  ]
 [ 0.    0.    0.    0.    0.25  0.  ]
 [ 0.    0.    0.    0.    0.    0.25]]
L (8, 8) = 
 [[ 0.25  0.   -0.25 -0.    0.25  0.   -0.25 -0.  ]
 [ 0.    0.25 -0.   -0.25  0.    0.25 -0.   -0.25]
 [-0.25 -0.    0.25  0.   -0.25 -0.    0.25  0.  ]
 [-0.   -0.25  0.    0.25 -0.   -0.25  0.    0.25]
 [ 0.25  0.   -0.25 -0.    0.25  0.   -0.25 -0.  ]
 [ 0.    0.25 -0.   -0.25  0.    0.25 -0.   -0.25]
 [-0.25 -0.    0.25  0.   -0.25 -0.    0.25  0.  ]
 [-0.   -0.25  0.    0.25 -0.   -0.25  0.    0.25]]
 --- Eigenvalues
lambda_0 = -0.000000+0.000000j
lambda_1 = -1.000000+0.000000j
lambda_2 = -0.000000+0.000000j
lambda_3 = -1.000000+0.000000j
lambda_4 = -0.000000+0.000000j
lambda_5 = 0.000000+0.000000j
lambda_6 = -0.000000+0.000000j
lambda_7 = -0.000000+0.000000j


In [14]:
## Debug --
sim_fr.check_kernel()

--- Kernel Eigenvectors
L@1_n^bar       =  [0. 0. 0. 0. 0. 0. 0. 0.]
L@p^*           =  [-0.  0. -0.  0.  0.  0.  0. -0.]
L@(I_mn)@p^*    =  [-0.  0. -0.  0.  0.  0.  0. -0.]
L@R(pi/4)@p^*   =  [-0. -0. -0. -0.  0.  0.  0.  0.]
L@Shx@p^*       =  [-0. -0. -0.  0.  0.  0.  0.  0.]
L@Shy@p^*       =  [-0.  0.  0.  0.  0.  0.  0. -0.]
 ------------ 


In [15]:
## Design the parameters of a body-fixed desired velocity in S #####

# Parameters examples ----------------------------------
# C1
params_deg_c1 = [1, 0, 0, 2, 1, 0]

# C2
params_deg_c2 = [0, 0, 1, 1, 2, 0]

# C3
params_deg_c3a = [1, 1, 0, 0, 0, 0]
params_deg_c3b = [1, 0, 0, 0, 4, 0]
params_deg_c3c = [0, 1, 0, 0, 0, 4]

# C4
params_deg_c4a = [1, 0, 0, 0, 0, 4]
params_deg_c4b = [0, 1, 0, 0, 4, 0]
# ------------------------------------------------------

# Set the parameters
vx, vy, a, omega, hx, hy = params_deg_c1

# Design the weights that gerenates the body-fixed desired velocity v_f^*
mu_matrix_x = np.array([[0,0,0,-1], [0,0,-1,0], [0,1,0,0], [1,0,0,0]])/2
mu_matrix_y = np.array([[0,-1,0,0], [1,0,0,0], [0,0,0,1], [0,0,-1,0]])/2

mu_matrix_a = np.array([[0,1,0,1], [1,0,1,0], [0,1,0,1], [1,0,1,0]])/2
mu_matrix_w = np.array([[0,1,0,-1], [-1,0,1,0], [0,-1,0,1], [1,0,-1,0]])/2
mu_matrix_hx = np.array([[0,0,0,1], [0,0,-1,0], [0,1,0,0], [-1,0,0,0]])/2
mu_matrix_hy = np.array([[0,1,0,0], [-1,0,0,0], [0,0,0,1], [0,0,-1,0]])/2

# Debug
sim_fr.set_manual_mu(vx*mu_matrix_x + vy*mu_matrix_y + a*mu_matrix_a + omega*mu_matrix_w + 
                     hx*mu_matrix_hx + hy*mu_matrix_hy)

sim_fr.check_M(eigenvectors=True, prec=[2,4,3])

M (8, 12) B.T (6, 4) K_inv (8, 8)
M^bar@B^T^bar@p_star: [ 2. -2.  0. -2.  0.  2.  2.  2.]

 -> L_mod
 --- Eigenvalues
lambda_0 = -0.000000+0.141421j
lambda_1 = -0.000000-0.141421j
lambda_2 = -4.000000+0.000000j
lambda_3 = -4.000000+0.000000j
lambda_4 = -0.000000+0.141421j
lambda_5 = -0.000000-0.141421j
lambda_6 = 0.000000+0.000000j
lambda_7 = 0.000000-0.000000j
--- Eigenvectors
v_0 = [ 0.612+0.j     0.   +0.j     0.204-0.289j  0.   +0.j    -0.204+0.289j
 -0.   -0.j     0.204+0.577j -0.   +0.j   ]
v_1 = [ 0.612-0.j     0.   -0.j     0.204+0.289j  0.   -0.j    -0.204-0.289j
 -0.   +0.j     0.204-0.577j -0.   -0.j   ]
v_2 = [-0.499+0.j -0.   +0.j  0.525+0.j  0.   +0.j -0.475+0.j -0.   +0.j
  0.499+0.j  0.   +0.j]
v_3 = [ 0.047+0.j -0.497+0.j -0.05 +0.j  0.523+0.j  0.045+0.j -0.473+0.j
 -0.047+0.j  0.497+0.j]
v_4 = [ 0.088+0.229j  0.187-0.529j  0.137+0.035j -0.187-0.264j -0.137-0.035j
  0.187+0.264j -0.186+0.16j   0.561+0.j   ]
v_5 = [ 0.088-0.229j  0.187+0.529j  0.137-0.035j -0.187+0.264j

In [16]:
affine_eig_R2(sim_fr, vx, vy, a, omega, hx, hy)

l+ = 0.14142135623730953j l- = -0.14142135623730953j
alfa, beta1, beta2 = -1.5 (0.5-1.4142135623730951j) (0.5+1.4142135623730951j)

----- Eigenvector of L_mod
(l1 != 0, l2 != 0) 

gamma1 = (0.4999999999999999+0.7071067811865474j)
gamma2 = (0.4999999999999999-0.7071067811865474j) 

v1(l+)_1 =  [ 0.577+0.204j  0.   +0.j     0.289-0.204j  0.   +0.j    -0.289+0.204j
  0.   +0.j    -0.   +0.612j  0.   +0.j   ]
v2(l+)_2 =  [ 0.   +0.j     0.577+0.204j  0.   +0.j     0.289-0.204j  0.   +0.j
 -0.289+0.204j  0.   +0.j    -0.   +0.612j]
v3(l-)_1 =  [ 0.577-0.204j  0.   +0.j     0.289+0.204j  0.   +0.j    -0.289-0.204j
  0.   -0.j    -0.   -0.612j  0.   -0.j   ]
v4(l-)_2 =  [ 0.   +0.j     0.577-0.204j  0.   +0.j     0.289+0.204j  0.   -0.j
 -0.289-0.204j  0.   -0.j    -0.   -0.612j] 

---- kappa*M@B.T@v
test-v1(l+)_1 | kappa*M@B.T: [-0.-0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
test-v2(l+)_2 | kappa*M@B.T: [ 0.+0.j -0.-0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
test-